In [3]:
# importing basic libraries
from santa_modelling import np, pd, plt, sns, setJupyterNotebook, title
from santa_modelling import Explore,Split,OHE,Balance,SetIndex, FeatureScale, FeatureScaleAll
from santa_modelling import ClassificationModelDictionary, RunAll,Classify
import santa_modelling

setJupyterNotebook()

In [2]:
import pickle
data = pickle.load(open('2020_Jan_13_22_56_41_dataImputedWithZero.pkl', 'rb'))

In [6]:
data[0].head()

,Age,Number_of_sexual_partners,First_sexual_intercourse,Num_of_pregnancies,Smokes,Smokes__years,Smokes__packs_per_year,Hormonal_Contraceptives,Hormonal_Contraceptives__years,IUD,...,STDs_Hepatitis_B,STDs_HPV,STDs__Number_of_diagnosis,Dx_Cancer,Dx_CIN,Dx_HPV,Dx,Hinselmann,Schiller,Citology
index,,,,,,,,,,,,,,,,,,,,,
709.000,25.000,2.000,16.000,1.000,0.000,0.000,0.000,1.000,9.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
516.000,20.000,2.000,16.000,1.000,0.000,0.000,0.000,1.000,1.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
786.000,25.000,6.000,17.000,2.000,0.000,0.000,0.000,1.000,2.000,0.000,...,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000
811.000,17.000,1.000,16.000,2.276,0.000,0.000,0.000,1.000,0.250,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
53.000,35.000,3.000,18.000,4.000,1.000,18.000,9.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [27]:
scores_selected=pd.read_csv('scores_ImputeWithZero.csv').head(10)
for_CV = {phase:list(scores_selected[scores_selected.Phase==phase].Model.unique())\
                  for phase in scores_selected.Phase.unique()}

In [28]:
models = ClassificationModelDictionary()
models = {models[i]['name']:[models[i]['model'],models[i]['cv_params'],models[i]['best_parameters']]for i in models\
         if models[i]['name'] in selected_models_list}
OHE_features = ['Dx:Cancer', 'Dx:CIN', 'Dx:HPV', 'Dx', 'Hinselmann', 'Schiller', 'Citology']

In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer,accuracy_score, recall_score
scoring = {'AUC': 'roc_auc', 'Accuracy': make_scorer(accuracy_score)}

for phase in for_CV.keys():
    if phase == 'unscaled':
        print(f'Selecting {phase} data');print(123*'*')
        unscaled_data = data
        for name in for_CV[phase]:
            model = models[name][0]
            parameters = models[name][1]
            print('Selecting Model:',name,'\nWith Param Dict:',parameters)

            cv = GridSearchCV(estimator=model, param_grid=parameters,cv=10,scoring=scoring,
                              refit='AUC',return_train_score=True)
            cv.fit(unscaled_data[0],unscaled_data[2])
            models[name][2][phase]=cv.best_params_
            print('Best Parameters: ------->',models[name][2]);print(123*'-')
    elif phase == 'ZscoreAll':
        print(f'Selecting {phase} data');print(123*'*')
        zcoreAll_data = FeatureScaleAll(data,scaler='StandardScaler')
        for name in for_CV[phase]:
            model = models[name][0]
            parameters = models[name][1]
            print('Selecting Model:',name,'\nWith Param Dict:',parameters)

            cv = GridSearchCV(estimator=model, param_grid=parameters,cv=10,scoring=scoring,
                              refit='AUC',return_train_score=True)
            cv.fit(zcoreAll_data[0],zcoreAll_data[2])
            models[name][2][phase]=cv.best_params_
            print('Best Parameters: ------->',models[name][2]);print(123*'-')
    elif phase == 'Zscore':
        print(f'Selecting {phase} data');print(123*'*')
        zcore_data = FeatureScale(data,OHE_features=OHE_features,scaler='StandardScaler')
        for name in for_CV[phase]:
            model = models[name][0]
            parameters = models[name][1]
            print('Selecting Model:',name,'\nWith Param Dict:',parameters)

            cv = GridSearchCV(estimator=model, param_grid=parameters,cv=10,scoring=scoring,
                              refit='AUC',return_train_score=True)
            cv.fit(zcore_data[0],zcore_data[2])
            models[name][2][phase]=cv.best_params_
            print('Best Parameters: ------->',models[name][2]);print(123*'-')
    elif phase == 'MinMaxAll':
        print(f'Selecting {phase} data');print(123*'*')
        minmaxAll_data = FeatureScaleAll(data,scaler='MinMaxScaler')
        for name in for_CV[phase]:
            model = models[name][0]
            parameters = models[name][1]
            print('Selecting Model:',name,'\nWith Param Dict:',parameters)

            cv = GridSearchCV(estimator=model, param_grid=parameters,cv=10,scoring=scoring,
                              refit='AUC',return_train_score=True)
            cv.fit(minmaxAll_data[0],minmaxAll_data[2])
            models[name][2][phase]=cv.best_params_
            print('Best Parameters: ------->',models[name][2]);print(123*'-')
    elif phase == 'MinMax':
        print(f'Selecting {phase} data');print(123*'*')
        minmax_data = FeatureScale(data,OHE_features=OHE_features,scaler='MinMaxScaler')
        for name in for_CV[phase]:
            model = models[name][0]
            parameters = models[name][1]
            print('Selecting Model:',name,'\nWith Param Dict:',parameters)

            cv = GridSearchCV(estimator=model, param_grid=parameters,cv=10,scoring=scoring,
                              refit='AUC',return_train_score=True)
            cv.fit(minmax_data[0],minmax_data[2])
            models[name][2][phase]=cv.best_params_
            print('Best Parameters: ------->',models[name][2]);print(123*'-')


Selecting MinMaxAll data
***************************************************************************************************************************
Applying MinMaxScaler
Selecting Model: LogisticRegression 
With Param Dict: {'penalty': ['l1', 'l2'], 'random_state': [0, 8]}
Best Parameters: -------> {'MinMaxAll': {'penalty': 'l1', 'random_state': 0}}
---------------------------------------------------------------------------------------------------------------------------
Selecting Model: SVC 
With Param Dict: {'C': [1, 10, 100, 500, 1000], 'kernel': ['rbf'], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001]}
Best Parameters: -------> {'MinMaxAll': {'C': 1, 'gamma': 1, 'kernel': 'rbf'}}
---------------------------------------------------------------------------------------------------------------------------
Selecting Model: GradientBoostingClassifier 
With Param Dict: {'loss': ['deviance', 'exponential'], 'n_estimators': [100], 'random_state': [0, 8]}
Best Parameters: -------> {'MinMaxAll': {'lo

In [30]:
cv_selected_models = {i:[models[i][0],models[i][2]] for i in models.keys() if models[i][2] != {}}
print('Best Models are'.center(100,'-'))
for key in cv_selected_models.keys():print(key,'with data',*cv_selected_models[key][1])

------------------------------------------Best Models are-------------------------------------------
LogisticRegression with data MinMaxAll MinMax unscaled Zscore
SVC with data MinMaxAll MinMax
BaggingClassifier with data MinMax
GradientBoostingClassifier with data MinMaxAll MinMax unscaled


In [31]:
scores = pd.DataFrame(columns=['Model','Phase','AUC_ROC','TrainingAccuracy','TestingAccuracy',
                               'Recall','Precision','F1_Score','FalsePositives','FalseNegatives'])
for key in cv_selected_models.keys():
    phases = list(cv_selected_models[key][1].keys())
    for phase in phases:
        if phase == 'unscaled':data_phase = data
        elif phase == 'ZscoreAll':data_phase = FeatureScaleAll(data,scaler='StandardScaler')
        elif phase == 'Zscore':data_phase = FeatureScale(data,OHE_features=OHE_features,scaler='StandardScaler')
        elif phase == 'MinMaxAll':data_phase = FeatureScaleAll(data,scaler='MinMaxScaler')
        elif phase == 'MinMax':data_phase = FeatureScale(data,OHE_features=OHE_features,scaler='MinMaxScaler')
        parameters = cv_selected_models[key][1][phase]
        model =  cv_selected_models[key][0]
        model.set_params(**parameters)
        scores = Classify(algorithm=key,model=model,data=data_phase,phase=phase,scores=scores)
scores.sort_values('AUC_ROC',ascending=False)

Applying MinMaxScaler
Applying MinMaxScaler
Applying StandardScaler
Applying MinMaxScaler
Applying MinMaxScaler
Applying MinMaxScaler
Applying MinMaxScaler
Applying MinMaxScaler


,Model,Phase,AUC_ROC,TrainingAccuracy,TestingAccuracy,Recall,Precision,F1_Score,FalsePositives,FalseNegatives
0,GradientBoostingClassifier,MinMaxAll,0.931,0.995,0.957,0.900,0.667,0.766,9,2
0,GradientBoostingClassifier,MinMax,0.931,0.995,0.957,0.900,0.667,0.766,9,2
0,GradientBoostingClassifier,unscaled,0.931,0.995,0.957,0.900,0.667,0.766,9,2
0,LogisticRegression,MinMaxAll,0.925,0.965,0.946,0.900,0.600,0.720,12,2
0,LogisticRegression,MinMax,0.925,0.965,0.946,0.900,0.600,0.720,12,2
0,LogisticRegression,Zscore,0.902,0.971,0.946,0.850,0.607,0.708,11,3
0,LogisticRegression,unscaled,0.900,0.964,0.942,0.850,0.586,0.694,12,3
0,BaggingClassifier,MinMax,0.835,0.999,0.950,0.700,0.667,0.683,7,6
0,SVC,MinMaxAll,0.729,0.986,0.922,0.500,0.500,0.500,10,10
0,SVC,MinMax,0.729,0.986,0.922,0.500,0.500,0.500,10,10


---

##### Final Model selected is GradientBoostingClassifier with Unscaled Data. Because with or without Gradient Boosting Classifer gives exact same performance
---

Selecting Model: GradientBoostingClassifier with Best Parameters: {'unscaled': {'loss': 'exponential', 'n_estimators': 100, 'random_state': 0}}